## Part 2B: Continuous shape analysis

In this notebook, we will learn the basics of shape analysis on parametric spline curves.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

### 0. Data loading

**0.1** Run the lines below to load the object models dataset.

In [ ]:
# BBBC010
directory = 'data/BBBC010/masks'
point_models=np.load(os.path.join(directory,'spline_models.npy'))

**0.2** Visualize a set of a few randomly selected models from the collection.